## SQL

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

### El objetivo principal de este proyecto es analizar una base de datos de libros, autores, editoriales y reseñas para:

- Entender el volumen de publicaciones recientes (post-2000)

- Analizar la interacción de los usuarios con los libros (reseñas y calificaciones)

- Identificar las editoriales y autores más relevantes según diferentes métricas

- Extraer insights sobre el comportamiento de los usuarios más activos

Este análisis ayudará a desarrollar una propuesta de valor para una nueva aplicación dirigida a amantes de la lectura.

In [32]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs',  # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode': 'require'})


### Se exploran las primeras 5 filas de cada tabla

In [33]:
query = "SELECT * FROM books LIMIT 5;"
pd.io.sql.read_sql(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [34]:
query = "SELECT * FROM authors LIMIT 5;"
pd.io.sql.read_sql(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [35]:
query = "SELECT * FROM publishers LIMIT 5;"
pd.io.sql.read_sql(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [36]:
query = "SELECT * FROM ratings LIMIT 5;"
pd.io.sql.read_sql(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [37]:
query = "SELECT * FROM reviews LIMIT 5;"
pd.io.sql.read_sql(query, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Se comienza a realizar las consultas objetivo

In [38]:
# Encuentra el número de libros publicados después del 1 de enero de 2000.
query = """SELECT COUNT(*) AS Numero_de_libros
FROM books
WHERE publication_date > '2000-01-01';"""
pd.io.sql.read_sql(query, con=engine)

,numero_de_libros
0,819


In [39]:
# Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
query = """SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    AVG(rt.rating) AS avg_rating
FROM 
    books b
LEFT JOIN 
    reviews r ON b.book_id = r.book_id
LEFT JOIN 
    ratings rt ON b.book_id = rt.book_id
GROUP BY 
    b.book_id, b.title
ORDER BY
    num_reviews DESC,
    avg_rating DESC;"""
pd.io.sql.read_sql(query, con=engine)

,book_id,title,num_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


In [ ]:
# Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
query = """SELECT 
    p.publisher,
    COUNT(b.book_id) AS num_books
FROM 
    books b
JOIN 
    publishers p ON b.publisher_id = p.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher
ORDER BY 
    num_books DESC
LIMIT 1;"""
print(pd.io.sql.read_sql(query, con=engine))

       publisher  num_books
0  Penguin Books         42


In [46]:
# Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
query = """SELECT 
    a.author,
    AVG(rt.rating) AS avg_rating
FROM 
    authors a
JOIN 
    books b ON a.author_id = b.author_id
JOIN 
    ratings rt ON b.book_id = rt.book_id
GROUP BY 
    a.author_id, a.author
HAVING 
    COUNT(rt.book_id) >= 50
ORDER BY 
    avg_rating DESC
LIMIT 1;"""
print(pd.io.sql.read_sql(query, con=engine))

           author  avg_rating
0  Diana Gabaldon         4.3


In [48]:
# Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.
query = """SELECT 
    AVG(user_reviews.count_reviews) AS avg_text_reviews
FROM (
    SELECT 
        username,
        COUNT(review_id) AS count_reviews
    FROM 
        reviews
    WHERE 
        username IN (
            SELECT username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50
        )
    GROUP BY 
        username
) AS user_reviews;"""
print(pd.io.sql.read_sql(query, con=engine))

   avg_text_reviews
0         24.333333


#### Conclusiones globales
- Encuentra el número de libros publicados después del 1 de enero de 2000.
  - Se encuentra que hay 819 libros publicados posterior al 1 de enero del 200.
- Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
  - Encontramos que los libros mas populares y mejor calificados pertenecen al genero juvenil (e.g. Twilight, Harry potter, etc.)
- Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
  - Econtramos que Penguin Books ha publicado 42 libros, siendo la editorial mas productiva
- Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones
  - Diana Gabaldon tiene un ranting promedio de 4.3, por lo que es la autora mejor calificada
- Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.
  - Observamos que el promedio de reseñas de texto es de 24.33 para los usuarios mas activos de la plataforma